# 05-build-model-TextCatCNN

## Main objectives:

- Use spaCy's neural netowkr model where token vectors are calculated using a CNN
    - [https://spacy.io/api/architectures#TextCatCNN](https://spacy.io/api/architectures#TextCatCNN)
- Use spaCy generated docs to train this model
- Run basic validation and evaluation 

In [1]:
import spacy
# load an english language model in spacy
nlp = spacy.load("en_core_web_lg")

In [2]:
!ls data

dev.spacy  train.spacy


# Validate configuration file

In [3]:
# validate configuration
!python -m spacy debug config ./config/config-TextCatCNN.cfg


============================= Config validation =============================

===================== Config validation for [initialize] =====================

====================== Config validation for [training] ======================
✔ Config is valid


In [4]:
!python -m spacy debug data ./config/config-TextCatCNN.cfg


============================ Data file validation ============================
✔ Corpus is loadable
✔ Pipeline can be initialized with data

=============================== Training stats ===============================
Language: en
Training pipeline: textcat
5000 training docs
5000 evaluation docs
✔ No overlap between training and evaluation data

============================== Vocab & Vectors ==============================
ℹ 282161 total word(s) in the data (18163 unique)
ℹ No word vectors present in the package

================================== Summary ==================================
✔ 3 checks passed


# Train the model

In [5]:
!python -m spacy train ./config/config-TextCatCNN.cfg --output ./models/textCatCNN

ℹ Using CPU

=========================== Initializing pipeline ===========================
[2021-03-22 22:42:07,593] [INFO] Set up nlp object from config
[2021-03-22 22:42:08,014] [INFO] Pipeline: ['textcat']
[2021-03-22 22:42:08,020] [INFO] Created vocabulary
[2021-03-22 22:42:08,020] [INFO] Finished initializing nlp object
[2021-03-22 22:42:18,341] [INFO] Initialized pipeline components: ['textcat']
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['textcat']
ℹ Initial learn rate: 0.001
wandb: W&B API key is configured (use `wandb login --relogin` to force relogin)
wandb: Tracking run with wandb version 0.10.23
wandb: Syncing run eager-hill-18
wandb: ⭐️ View project at https://wandb.ai/garza/yelp-polarity
wandb: 🚀 View run at https://wandb.ai/garza/yelp-polarity/runs/1qd1puec
wandb: Run data is saved locally in /home/garza/notebooks/aicamp/yelp-project/wandb/run-20210322_224223-1qd1puec
wandb: Run `wandb offline` to tur

# Evaluate our best model output and save metrics to disk

For hyperparameter tuning, experimented with different tok2vec attributes and row sizes settling on:

- "NORM", "PREFIX", "SUFFIX", "SHAPE"
- [10000, 5000, 5000, 5000]

Also adjusted the width size from 64 to 96, which only resulted in a nominal increase in performance.

Training was tested on training datasets of 100, 500, 1000 and finally, 5000.

In [6]:
!python -m spacy evaluate ./models/textCatCNN/model-best ./data/dev.spacy --output ./evaluate/model-textCatCNN-metrics.json

ℹ Using CPU

================================== Results ==================================

TOK                   100.00
TEXTCAT (macro AUC)   93.04 
SPEED                 17016 


=========================== Textcat F (per label) ===========================

               P       R       F
positive   86.45   84.44   85.43
negative   84.79   86.76   85.77


======================== Textcat ROC AUC (per label) ========================

           ROC AUC
positive      0.93
negative      0.93

✔ Saved results to evaluate/model-textCatCNN-metrics.json


In [7]:
!python -m spacy debug data ./config/config-TextCatCNN.cfg 


============================ Data file validation ============================
✔ Corpus is loadable
✔ Pipeline can be initialized with data

=============================== Training stats ===============================
Language: en
Training pipeline: textcat
5000 training docs
5000 evaluation docs
✔ No overlap between training and evaluation data

============================== Vocab & Vectors ==============================
ℹ 282161 total word(s) in the data (18163 unique)
ℹ No word vectors present in the package

================================== Summary ==================================
✔ 3 checks passed
